# Setup

## Installs

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import gpxpy
import folium

In [2]:
GPX_FILE_PATH = '../gpx/'
HARD_SLOPE_THRESHOLD = 0.2 # Threshold to consider a section as hard

colors = {
    'main': '#f53b57',
    'secondary': '#3c40c6',
    'tertiary': '#ffdd59',
}

In [3]:
plt.style.use('ggplot')

# Processing the GPX files

In [26]:
# Read the GPX file
gpx_file_name = 'ktr-campos-27-2024.gpx'
full_gpx_file_path = os.path.join(GPX_FILE_PATH, gpx_file_name)
gpx_file = gpxpy.parse(open(full_gpx_file_path))

# Only extract the first track and segment
track = gpx_file.tracks[0]
segment = track.segments[0]
points = segment.points

# Extract information from points
coordinates = np.array([[point.latitude, point.longitude, point.elevation] for point in points]) #@!delete
df = pd.DataFrame(
    coordinates,
    columns=['latitude', 'longitude', 'elevation']
)

# Calculate the center of the route (based on the df)
center_coordinates = np.mean(
    df[['latitude', 'longitude']].values,
    axis=0
)

# Get basic statistics of the route
total_distance = segment.length_3d() # Total distance in meters
total_elevation_gain = segment.get_uphill_downhill().uphill
total_elevation_loss = segment.get_uphill_downhill().downhill

print(f"Total distance: {total_distance/1000:.1f} km")
print(f"Total elevation gain: {round(total_elevation_gain):,} m")
print(f"Total elevation loss: {round(total_elevation_loss):,} m")
print(f"Total elevation gain / distance: {total_elevation_gain/total_distance:.3f}")

# Bounds of the route
min_lat = df['latitude'].min()
max_lat = df['latitude'].max()
min_lon = df['longitude'].min()
max_lon = df['longitude'].max()

# Create map with auto zoom based on bounds
map = folium.Map(
    location=center_coordinates,
    zoom_start=12,
)

# Fit bounds to the route
map.fit_bounds(
    [
        [min_lat, min_lon],  # Southwest corner
        [max_lat, max_lon]   # Northeast corner
    ]
)

# Add the route to the map
folium.PolyLine(
    df[['latitude', 'longitude']].values,
    weight=5,
    color=colors['main'],
    opacity=0.8
).add_to(map)

# Add the start and end points to the map
folium.Marker(
    location=df.iloc[0][['latitude', 'longitude']].values,
    icon=folium.Icon(color='green', icon='circle'),
).add_to(map)

folium.Marker(
    location=df.iloc[-1][['latitude', 'longitude']].values,
    icon=folium.Icon(color='red', icon='circle'),
).add_to(map)

# Display the map
map

Total distance: 25.8 km
Total elevation gain: 1,564 m
Total elevation loss: 1,566 m
Total elevation gain / distance: 0.061


In [ ]:
elevations = coordinates[:, 2]
elevation_diff_between_points = np.diff(elevations)

# Calculate the cumulative distance between each point on the route
distance_between_points_3d = np.zeros(len(points) - 1)
distance_between_points_2d = np.zeros(len(points) - 1)

for i in range(0, len(points) - 1):
    distance_between_points_3d[i] = gpxpy.geo.distance(
        coordinates[i, 0],  # latitude1
        coordinates[i, 1],  # longitude1
        coordinates[i, 2],  # elevation1
        coordinates[i+1, 0],   # latitude2
        coordinates[i+1, 1],   # longitude2
        coordinates[i+1, 2],    # elevation2
    )

    distance_between_points_2d[i] = gpxpy.geo.distance(
        coordinates[i, 0],  # latitude1
        coordinates[i, 1],  # longitude1
        0, # Set elevation to 0 for 2D distance
        coordinates[i+1, 0],   # latitude2
        coordinates[i+1, 1],   # longitude2
        0, # Set elevation to 0 for 2D distance
    )

# Calculate cumulative distance and convert to km
cum_distance_3d = np.cumsum(distance_between_points_3d)
cum_distance_3d = cum_distance_3d / 1000

# Calculate slope gradients
slope_gradients = elevation_diff_between_points / distance_between_points_2d

# Create the plot
fig = px.line(
    x=cum_distance_3d,
    y=elevations[:-1],
    title='Elevation Profile',
    labels={
        'x': 'Distance (km)',
        'y': 'Elevation (m)'
    }
)

fig.update_traces(
    line_color=colors['main'],
    line_width=2,
    fill='tonexty',
    fillcolor=f"rgba{tuple(int(colors['main'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4)) + (0.1,)}",
    mode='lines',
)

fig.update_layout(
    showlegend=False,
    plot_bgcolor='white',
    width=900,
    height=400,
    hoverdistance=100,     # Maximum distance to show hover effect
    spikedistance=100,     # Maximum distance to show spike
    yaxis=dict(
        range=[min(elevations[:-1]) - 50, max(elevations[:-1]) + 50],
        tickformat='.0f',  # Format y-axis ticks to 1 decimal place
        showline=True,
        showgrid=True,
        gridcolor='lightgrey',
        showspikes=True,         # Show spike line
        spikecolor=colors['main'],
        spikesnap='data',      # Spike snaps to data points
        spikemode='across',      # Spike goes across the plot
        spikethickness=1
    ),
    xaxis=dict(
        tickformat='.1f',  # Format x-axis ticks to 2 decimal places
        showline=True,
        showgrid=True,
        gridcolor='lightgrey',
        spikemode='across',
        spikesnap='data',
        spikethickness=1,
        spikecolor=colors['main']
    ),
    yaxis_title='Elevation (m)',
    xaxis_title='Distance (km)',
)

fig.show()

In [ ]:
# Plot the slope gradients
plt.figure(figsize=(12, 6))
plt.plot(
    cum_distance_3d,
    slope_gradients,
    color=colors['main'],
    linewidth=2
)
plt.xlabel('Distance (km)')
plt.ylabel('Slope Gradient')
plt.title('Slope Gradient Profile')
plt.grid(True)
plt.show()

In [ ]:
# Define the slope bins and labels
slope_bins = np.arange(-1, 1, 0.05).round(2)
slope_bin_labels = [f"{round(100*slope_bins[i])} → {round(100*slope_bins[i+1])}%" for i in range(len(slope_bins) - 1)]

# Make a Pandas DF with the data
df = (
    pd.DataFrame({
        'distance': cum_distance_3d,
        'elevation': elevations[:-1],
        'slope_gradient': slope_gradients
    })
    .assign(
        # Bin the slope gradient into 5% bins
        slope_bin=lambda x: pd.cut(
            x['slope_gradient'] + 0.0001,
            bins=slope_bins,
            right=False,
            labels=slope_bin_labels
        ),
        hard_slope=lambda x: abs(x['slope_gradient']) > HARD_SLOPE_THRESHOLD
    )
)

df

In [ ]:
# Plot a histogram of the slope classifications
slope_df = (
    df
    .groupby('slope_bin')
    .size()
)

# % of hard slopes
hard_slope_perc = (
    (
        df
        .groupby('hard_slope')
        .size()/len(df)
    )
    .loc[True]
)

print(f"% of slopes with gradient > {HARD_SLOPE_THRESHOLD}: {100*hard_slope_perc:.1f}%")
# Plot the histogram
(
    slope_df
    .plot(
        kind='bar',
        color=colors['main'],
        figsize=(12, 6),
    )
);